### Random Forest Model ###

In [2]:
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification
from pyspark.sql import functions as fn, Row
from pyspark import sql
import re

import matplotlib.pyplot as plt
import pandas as pd

In [3]:
flights_rf_clean = pd.read_csv('flights_indexed_clean.csv')
flights_rf_clean = flights_rf_clean.drop(columns=['CANCELLED', 'Unnamed: 0'])

In [4]:
flights_rf_clean = spark.createDataFrame(flights_rf_clean)

In [5]:
rf_training_df, rf_validation_df, rf_testing_df = flights_rf_clean.randomSplit([0.6, 0.3, 0.1], seed=0)

In [6]:
# Random Forest Pipeline #

## EXPERIMENT:  See the pipeline section above ##

pipe_randomforest = Pipeline(stages = [
    feature.VectorAssembler(inputCols=['DIVERTED',
 'AIRLINE_index',
 'FLIGHT_NUMBER_index',
 'TAIL_NUMBER_index',
 'ORIGIN_AIRPORT_index',
 'DESTINATION_AIRPORT_index',
 'SCHEDULED_DEPARTURE_index',
 'SCHEDULED_TIME_index',
 'SCHEDULED_ARRIVAL_index'], outputCol='features',),
    classification.RandomForestClassifier(numTrees=20, maxDepth=13, featuresCol='features',labelCol='DEPARTURE_DELAY_index',)\
    .setSeed(0)
]).fit(rf_training_df)

In [7]:
randomforest_df = pipe_randomforest.transform(rf_validation_df).toPandas()

In [8]:
# Create dataframe which contains the random forest predictions, as well as the actual label,
# and then build a confusion matrix #

evaluation_df = pd.concat([randomforest_df['prediction'],randomforest_df['DEPARTURE_DELAY_index']], 
                          axis = 1, keys=['prediction','label'])
crosstab_df = pd.crosstab(evaluation_df.prediction, evaluation_df.label)
display(crosstab_df)
print("Accuracy: {:,.3f}%" .format((crosstab_df[0][0] + crosstab_df[1][1]) / crosstab_df.values.sum() * 100))

# Less accurate than our previous model, but that would make sense because it has less information available #

label,0.0,1.0
prediction,,
0.0,14030,6950
1.0,2257,3066


Accuracy: 64.996%


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 36452)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp